In [1]:
import folium
from folium import IFrame
import json
import pandas as pd
import geopandas as gpd

In [2]:
df = pd.read_csv('areas_by_state.csv', index_col=0)

In [3]:
wildfires_df = pd.read_csv('wildfires_clean_season.csv', index_col=0)

/home/linuxbrew/.linuxbrew/opt/python/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Folium does not like indexes and wants a column name. Resetting index.

In [ ]:
df.reset_index(inplace=True)

Loading geojson.

In [5]:
us_geojson = json.load(open('us_geojson_features.json'))

Creating a folium map object centered on the US.

In [12]:
m = folium.Map(location=(40.373471, -95.843056), zoom_start=4)

Creating a new column containing the number of fires per square km.

In [50]:
data = pd.DataFrame(wildfires_df.groupby('state').count()['id'])

In [53]:
data['fires_per_sqkm'] = data.id/surface_area

In [55]:
data['fires_per_sqkm'] = data['fires_per_sqkm'].map('{:.2f}'.format)

In [57]:
data.reset_index(inplace=True)

In [59]:
data['fires_per_sqkm'] = pd.to_numeric(data.fires_per_sqkm)

In [63]:
data.to_csv('fires_per_sqkm.csv')

In [4]:
data = pd.read_csv('fires_per_sqkm.csv')

Creating a choropleth folium object that displays the variable data (that is the number of fires per sqkm) and adds it to the map object through the .add_to method.

In [13]:
choro_summer = folium.Choropleth(
    geo_data=us_geojson,
    name='Summer',
    data=data,
    columns=['state', 'fires_per_sqkm'],
    key_on='feature.id',
    fill_color='OrRd',
    nan_fill_color='red',
    fill_opacity=1,
    line_opacity=1,
    hightlight=True,
    legend_name='Fires per square km.'
).add_to(m)

Creating a GeoJsonTooltip object.

In [14]:
tooltip_summer=folium.features.GeoJsonTooltip(
            fields=['name', 'fires_per_sqkm'],
            aliases=['State: ', 'Fires per square km.'])

Adding the Tooltips to the map (m) object

In [15]:
tooltip_summer.add_to(choro_summer.geojson)

In [19]:
m

Saving it in html format.

In [20]:
m.save('index.html')